## Import packages

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 1s 1us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


# Process the text

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(50)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  'H' :  20,
  'I' :  21,
  'J' :  22,
  'K' :  23,
  'L' :  24,
  'M' :  25,
  'N' :  26,
  'O' :  27,
  'P' :  28,
  'Q' :  29,
  'R' :  30,
  'S' :  31,
  'T' :  32,
  'U' :  33,
  'V' :  34,
  'W' :  35,
  'X' :  36,
  'Y' :  37,
  'Z' :  38,
  'a' :  39,
  'b' :  40,
  'c' :  41,
  'd' :  42,
  'e' :  43,
  'f' :  44,
  'g' :  45,
  'h' :  46,
  'i' :  47,
  'j' :  48,
  'k' :  49,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


# The prediction task

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [10]:
for i in char_dataset.take(10):
    print(i.numpy())

18
47
56
57
58
1
15
47
58
47


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


## Create training batches

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

# Build The Model

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

# Try the model

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[[-2.48086732e-03  1.00823827e-02  1.13818590e-02 ...  3.05997115e-03
    4.07742010e-03 -1.05883852e-02]
  [ 5.99613297e-04  2.15811990e-02 -3.25755170e-03 ... -1.03496239e-02
   -5.05751092e-03 -9.13000759e-03]
  [ 1.18761638e-03  1.18700573e-02  3.50734126e-03 ... -1.02916593e-03
   -1.02098752e-03 -1.62298828e-02]
  ...
  [ 1.04224812e-02  3.39803472e-03  2.98181688e-03 ... -1.16551267e-02
   -1.50435427e-02  1.09347291e-02]
  [ 7.82824866e-03 -3.96170001e-03  7.95410387e-03 ...  8.84433370e-03
   -1.27658173e-02 -1.85273297e-03]
  [-6.33857027e-03 -1.00535760e-02  5.80833899e-03 ...  1.06511265e-02
   -3.73602449e-03 -4.40566242e-03]]

 [[-2.17783032e-03  8.54960177e-04 -1.41076753e-02 ... -2.62381951e-03
    6.90550637e-03 -9.70943458e-03]
  [ 5.17665781e-03 -3.56756384e-04 -8.55453778e-03 ...  1.45255364e-02
    5.42390347e-03 -5.69553161e-03]
  [ 2.13938970e-02 -3.31166433e-04 -4.68946900e-03 ...  7.72979762e-03
    3.63833550e-03 -8.55874550e-03]
  ...
  [ 9.738591

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([41, 16, 61, 27, 58,  1, 13, 38, 33, 45, 37, 41, 17, 21, 16, 23, 62,
       10, 33, 35, 17, 57, 56, 32, 31, 12, 51, 45, 46, 63, 28, 51, 47, 29,
       19, 20, 35, 27, 41, 44, 15,  9, 55, 50, 27, 13, 48, 51,  6, 34, 52,
       39, 43, 18, 50,  8, 33, 59, 44, 56, 19, 23, 19, 42, 59, 11,  3, 32,
        1, 34, 43,  9, 53, 63, 35, 63,  5, 11, 56,  4, 47,  9, 54, 20, 42,
       26, 23, 22,  9, 30, 60, 32, 46, 39, 20, 38, 30, 29, 31,  3],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "el's monument.\n\nBALTHASAR:\nIt doth so, holy sir; and there's my master,\nOne that you love.\n\nFRIAR LA"

Next Char Predictions: 
 "cDwOt AZUgYcEIDKx:UWEsrTS?mghyPmiQGHWOcfC3qlOAjm,VnaeFl.UufrGKGdu;$T Ve3oyWy';r&i3pHdNKJ3RvThaHZRQS$"


# Train the model

In [24]:
# Attach an optimizer, and a loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174078


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Execute the training

In [29]:
EPOCHS = 10

In [30]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 1081s 6s/step - loss: 2.4212
Epoch 2/10
172/172 [==============================] - 847s 5s/step - loss: 1.9083
Epoch 3/10
172/172 [==============================] - 799s 5s/step - loss: 1.6612
Epoch 4/10
172/172 [==============================] - 749s 4s/step - loss: 1.5245
Epoch 5/10
172/172 [==============================] - 695s 4s/step - loss: 1.4429
Epoch 6/10
172/172 [==============================] - 686s 4s/step - loss: 1.3873
Epoch 7/10
172/172 [==============================] - 793s 5s/step - loss: 1.3415
Epoch 8/10
172/172 [==============================] - 662s 4s/step - loss: 1.3033
Epoch 9/10
172/172 [==============================] - 796s 5s/step - loss: 1.2704
Epoch 10/10
172/172 [==============================] - 796s 5s/step - loss: 1.2380


## Generate text

In [31]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [32]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [34]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 2000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model, start_string=u"Q"))

QUYYOUS:
Varrier, nollibe,
Confiration shoes with such a tobbling
By: I be
Satisf,' sands, 'War ience will be to her;
To tell what I prove all, daughter.

LUCIO:
'Tis a visite live, ir hatcheghat a pretty pirce;
But now the peace of suafty ragler, I
descing it contrive, to father is emploces:
Tabeak up two leds that leave your foil:
If he quit Edward will both make two brother;
So stoow the nobleside to tear from him,
To make this lack of answer,
his lips blood is like no rem mine.

MENENIUS:
peace;
To hell for this young guilty of
thy father should I wonder,
But in the worfard to venge of twail.

LUCIO:
Our roy is a notething, lie know me
To stille my honour's love, and safe.

CORIOLANUS:
Master came to Prince's blush!

GLOUCESTER:

KING RICHARD III:
Bragge your beholding to thy certions
Have bees deaves done so love a flower.

GLOUCESTER:
Camillo, My fortune constast what kissmy too?

JULIET:
The tribunes. Now he could scarce to see me
Even bows, Claudis't be teeming, I have beent
fo